In [1]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# Building a Neural Network with Keras

In [2]:
import gzip
import pickle
import numpy  as np
import keras
import tensorflow as tf

Using TensorFlow backend.


The following magic command is necessary to prevent the Python kernel to die because of linkage problems.

In [3]:
%env KMP_DUPLICATE_LIB_OK=TRUE

env: KMP_DUPLICATE_LIB_OK=TRUE


The function $\texttt{vectorized_result}(d)$ converts the digit $d \in \{0,\cdots,9\}$ and returns a NumPy vector $\mathbf{x}$ of shape $(10, 1)$ such that
$$
\mathbf{x}[i] = 
\left\{
  \begin{array}{ll}
     1 & \mbox{if $i = j$;} \\
     0 & \mbox{otherwise.}
  \end{array}  
\right.
$$
This function is used to convert a digit $d$ into the expected output of a neural network that has an output unit for every digit.

In [4]:
def vectorized_result(d):
    e    = np.zeros((10, ), dtype=np.float32)
    e[d] = 1.0
    return e

The function $\texttt{load_data}()$ returns a pair of the form
$$ (\texttt{training_data}, \texttt{test_data}) $$
where 
<ul>
<li> $\texttt{training_data}$ is a list containing 60,000 pairs $(\textbf{x}, \textbf{y})$ s.t. $\textbf{x}$ is a 784-dimensional `numpy.ndarray` containing the input image and $\textbf{y}$ is a 10-dimensional `numpy.ndarray` corresponding to the correct digit for x.</li>
<li> $\texttt{test_data}$ is a list containing 10,000 pairs $(\textbf{x}, y)$.  In each case, 
     $\textbf{x}$ is a 784-dimensional `numpy.ndarry` containing the input image, 
     and $y$ is the corresponding digit value.
</ul>

In [5]:
def load_data():
    with gzip.open('../mnist.pkl.gz', 'rb') as f:
        train, validate, test = pickle.load(f, encoding="latin1")
    X_train = np.array([np.reshape(x, (784, )) for x in train[0]])
    X_test  = np.array([np.reshape(x, (784, )) for x in test [0]])
    Y_train = np.array([vectorized_result(y) for y in train[1]])
    Y_test  = np.array([vectorized_result(y) for y in test [1]])
    return (X_train, X_test, Y_train, Y_test)

In [6]:
X_train, X_test, Y_train, Y_test = load_data()

Let us see what we have read:

In [7]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((50000, 784), (10000, 784), (50000, 10), (10000, 10))

Below, we create a neural network with two hidden layers.
- The first hidden layer has 60 nodes and uses the <a href="https://en.wikipedia.org/wiki/Rectifier_(neural_networks)">ReLU function</a> 
  as activation function.
- The second hidden layer uses 30 nodes and also uses the ReLu function.  
- The output layer uses the <a href="https://en.wikipedia.org/wiki/Softmax_function">softmax function</a> as 
  activation function.  This function is defined as follows:
  $$ \sigma(\mathbf{z})_i := \frac{e^{z_i}}{\sum\limits_{d=0}^{9} e^{z_d}}  $$
  Here, $N$ is the number of output nodes and $z_i$ is the sum of the inputs of the $i$-th output neuron.
  This function guarantees that the outputs of the 10 output nodes can be interpreted as probabilities, since 
  there sum is equal to $1$.
- The <em style="color:blue">loss function</em> used is the <em style="color:blue">cross-entropy</em>.  
  If a neuron outputs the value $a$, when it should output the value $y \in \{0,1\}$, the cross entropy cost of 
  this neuron is defined as
  $$ C(a, y) := - y \cdot \ln(a) - (1-y)\cdot \ln(1-a). $$
- The cost function is minimzed using stochastic gradient descent with a learning rate of $0.3$.

In [8]:
model = keras.models.Sequential()
model.add(keras.layers.Dense( 80, activation='relu', input_dim=784))
model.add(keras.layers.Dense( 40, activation='relu'               ))
model.add(keras.layers.Dense( 40, activation='relu'               ))
model.add(keras.layers.Dense( 10, activation='softmax'            ))
model.compile(loss       = 'categorical_crossentropy', 
              optimizer  = tf.keras.optimizers.SGD(lr=0.3), 
              metrics    = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 80)                62800     
_________________________________________________________________
dense_2 (Dense)              (None, 40)                3240      
_________________________________________________________________
dense_3 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                410       
Total params: 68,090
Trainable params: 68,090
Non-trainable params: 0
_________________________________________________________________


In [9]:
%%time
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30, batch_size=100, verbose=1)

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 2s 46us/step - loss: 0.4084 - accuracy: 0.8748 - val_loss: 0.1763 - val_accuracy: 0.9508
Epoch 2/30
50000/50000 [==============================] - 2s 40us/step - loss: 0.1439 - accuracy: 0.9557 - val_loss: 0.1265 - val_accuracy: 0.9621
Epoch 3/30
50000/50000 [==============================] - 2s 38us/step - loss: 0.1035 - accuracy: 0.9684 - val_loss: 0.1056 - val_accuracy: 0.9671
Epoch 4/30
50000/50000 [==============================] - 2s 40us/step - loss: 0.0807 - accuracy: 0.9752 - val_loss: 0.0959 - val_accuracy: 0.9713
Epoch 5/30
50000/50000 [==============================] - 2s 44us/step - loss: 0.0660 - accuracy: 0.9801 - val_loss: 0.1055 - val_accuracy: 0.9691
Epoch 6/30
50000/50000 [==============================] - 2s 43us/step - loss: 0.0554 - accuracy: 0.9828 - val_loss: 0.1010 - val_accuracy: 0.9703
Epoch 7/30
50000/50000 [==============================] - 2s 42us/st